In [ ]:
import pandas as  pd

In [ ]:
d=pd.read_csv("./Medicaid1986.csv")
d.drop(["health1","health2"], axis=1, inplace=True)
d.head()

In [ ]:
bins = {
    'visits': [-1, 1, 6, 10, float('inf')],
    'exposure': [0, 30, 60, 90, 120, float('inf')],
    'children': [-1, 0, 1, 2, 3, float('inf')],
    'age': [0, 18, 30, 50, 70, float('inf')],
    'income': [-1, 30, 50, 70, 90, float('inf')],
    'access': [ 0, 1, float('inf')],

}

labels = {
    'visits': ['0-1', 'between 2 and 6', 'between 7 and 10', '10+'],
    'exposure': ['0-30', '31-60', '61-90', '91-120', '120+'],
    'children': ['None', 'One', 'Two', 'Three', 'Four or more'],
    'age': ['Child', 'Young_Adult', 'Adult', 'Middle-Aged', 'Senior'],
    'income': ['Low', 'Lower_Middle', 'Middle', 'Upper_Middle', 'High'],
    'access': ['Low_Access', 'High_Access'],

}

for feature in bins.keys():
    d[feature + '_category'] = pd.cut(d[feature], bins=bins[feature], labels=labels[feature])

d.drop(list(bins.keys()), axis=1, inplace=True)

In [ ]:
d.drop(["school","rownames"], axis=1, inplace=True)
d.head()

In [ ]:
d.to_csv("test2.csv", index=False)

In [ ]:
ap=pd.DataFrame()

In [ ]:
for col in d.columns:
    uniqitems=d[col].unique()
    for uniq in uniqitems:    
        ap[col+"_"+str(uniq)]= [1 if (str(value) == str(uniq)) else 0 for value in d[col]]

In [ ]:
# ap.to_csv("test.csv", index=False)

In [ ]:
ap

In [ ]:
# minSup= 498/996

def L1(ap,minSup):
    my_series=ap.sum(axis=0)
    l1=my_series[my_series > minSup]
    print(l1)
    print(len(l1))
    return l1,l1.index


In [ ]:
l1 , nw_col =L1(ap,498)
new_ap =ap[nw_col]
new_ap

In [ ]:
def ItemSet(itemSet_p,Minsup,data):
    list1=[]
    list2=[]
    for index, col1 in enumerate(itemSet_p.index[:-1]):
        for ind, col2 in enumerate(itemSet_p.index[index+1:]):
            if(col1.rsplit("+",1)[0]==col2.rsplit("+",1)[0]):
                new_col=col1 +"+"+ col2.rsplit("+",1)[-1]
                list1.append(new_col) 
                columns=new_col.split("+")
                condition = (data[columns[0]] == 1)
                for i in (columns[1:]):
                    condition &= (data[i] == 1)
                list2.append(len(data[condition]))
                
    new_SR=pd.Series(data=list2,
    index=list1)
    new_SR=new_SR[new_SR > Minsup]

    return new_SR

In [ ]:
def Apriori(Minsup,data):
    my_series=ap.sum(axis=0)
    sr=my_series[my_series > Minsup]
    disct={"1_itemsets":sr}
    print("1_itemsets")
    print(l1)
    print(len(l1))
    print("\n")
    
    list1=[]
    list2=[]
    
    for index, col in enumerate(sr.index[:-1]):
        for ind, col2 in enumerate(data.columns[index+1:]):
            list1.append(col + "+" + col2 )
            list2.append( len(data[(data[col] == 1) & (data[col2] == 1)]))

    sr=pd.Series(data=list2,
        index=list1)
    sr=sr[sr > Minsup]
    sr1=sr
    niveau=2

    while(len(sr1)!=0):
        print(f"{niveau}_itemsets")
        print(sr1)
        print(len(sr1))
        print("\n")
        disct[f"{niveau}_itemsets"]=sr1
        sr1=ItemSet(sr1,Minsup,data)
        niveau+=1

    return disct



    

#l'idea est change le minsup minf----minsup----maxf ====> s9si ismail (decotomique)
#3la ap ji kter mn new_ap

In [ ]:
dic =Apriori(498,new_ap)

In [ ]:
from itertools import combinations

def sup(item_set):
    """
    jib sup in dictionary dic
    """
    key=f"{len(item_set.split('+'))}_itemsets"

    return dic[key][item_set] 

def construire_regles(Item_set):
    Confiance=[] 
    regles = []
    items = Item_set.split("+")
    for r in range(1, len(items)):
        for comb in combinations(items, r):
            left = "+".join(list(comb))
            right = "+".join([item for item in items if item not in comb])
            regles.append(left + "->" + right)
            # ////
            sup_Antecedant=sup(Item_set)
            sup_Consequent=sup(left)
            Confiance.append(sup_Antecedant/sup_Consequent)

            
    return regles,Confiance



In [ ]:
regle=[]
Concfiance=[]

for key in list(dic.keys())[1:]:
    for itemset in dic[key].index:
        regles,Concfiances= construire_regles(itemset)

        regle=regle+regles
        Concfiance=Concfiance+Concfiances

SR=pd.Series(Concfiance, index=regle)
SR=SR[SR>=0.65]
SR.name="Concfiance"
SR.index.name="Règles d'association"
SR.to_csv("Regles.csv")

In [ ]:
# pour l'optimistaion
"""
from multiprocessing import Pool
from joblib import Parallel, delayed
import dask

# 1- Calcul du support
def calculate_support(itemset):
    # Insérez ici le code pour calculer le support d'un itemset
    pass

with Pool() as p:
    supports = p.map(calculate_support, itemsets)

# 2- Génération des règles d'association
def generate_association_rules(itemset):
    # Insérez ici le code pour générer les règles d'association d'un itemset
    pass

association_rules = Parallel(n_jobs=-1)(delayed(generate_association_rules)(itemset) for itemset in itemsets)

# 3- Calcul de la confiance
def calculate_confidence(rule):
    # Insérez ici le code pour calculer la confiance d'une règle d'association
    pass

confidence_values = dask.compute(*[dask.delayed(calculate_confidence)(rule) for rule in association_rules])
"""